# Логика

In [ ]:
import numpy as np
import torch

In [ ]:
import sys
sys.path.insert(0, './python')

In [ ]:
from fuzzy_torch import logic

## Градиенты

In [ ]:
a = torch.tensor(np.array([0.5, 0.9]), requires_grad=True)
b = torch.tensor(np.array([0.7, 0.8]), requires_grad=True)

In [ ]:
d = torch.sum(logic.Product.fuzzy_or(a, b))

In [ ]:
d.backward()

In [ ]:
b.grad

## Индикаторные функции

In [ ]:
from fuzzy_torch.layers.indicators import Sigmoid, Gaussian

In [ ]:
params = Sigmoid((2))

In [ ]:
params.linear.weight = torch.nn.Parameter(torch.tensor([[0.5, 0.5]]))
params.linear.bias = torch.nn.Parameter(torch.tensor([-0.5]))

In [ ]:
print(params.linear.weight)
print(params.linear.bias)

In [ ]:
params(torch.log(torch.tensor([[0.0000001, 1.0], [200.0, 0.1]])))

In [ ]:
(torch.zeros(2) + 0.0001) / (torch.zeros(2) + 0.0001)

### Простейшие логические выражения

In [ ]:
N_temp = 101
N_wind = 101

temperature_np = np.linspace(-40.0, 60.0, N_temp)
wind_speed_np = np.linspace(0.0, 10, N_wind)

# Сетка значений.
temperature_np_grid, wind_speed_np_grid = np.meshgrid(temperature_np, wind_speed_np)
temperature_np_grid = temperature_np_grid.flatten()[:,None].astype(np.float32)
wind_speed_np_grid = wind_speed_np_grid.flatten()[:,None].astype(np.float32)

In [ ]:
temperature = torch.tensor(temperature_np_grid, requires_grad=True)
wind_speed  = torch.tensor(wind_speed_np_grid, requires_grad=True)

In [ ]:
# Логические переменные.
wind_speed_is_high  = Gaussian(1, weight=8.0, offset=1.0)
temperature_is_low  = Sigmoid(1, weight=-0.1, offset=5.0)
temperature_is_high = Sigmoid(1, weight=0.1, offset=35.0)

In [ ]:
# Логика.
#Logic = logic.Godel
#Logic = logic.Product
#Logic = logic.Lukasiewicz
Logic = logic.Hamacher

# Алиасы для оераций.
fzand  = Logic.fuzzy_and
fzor   = Logic.fuzzy_or
fzimpl = Logic.fuzzy_impl

# Логическое выражение.
close_window = fzor(
    wind_speed_is_high(torch.log10(wind_speed)),
    fzor(
        temperature_is_low(temperature),
        temperature_is_high(temperature)
    )
)

In [ ]:
close_window_np = close_window.detach().numpy()[:,0].reshape((N_temp, N_wind))

In [ ]:
import matplotlib.pyplot as pl

fig = pl.figure()
ax = fig.gca()
cfset = ax.contourf(temperature_np, wind_speed_np, close_window_np, cmap='Blues', levels=100)
ax.set_xlabel('temp')
ax.set_ylabel('wind')